<a href="https://colab.research.google.com/github/leadtech85/3.7-DigiPrime/blob/main/category_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Category Mapping POC

## Preparation Zero: Setup

*   Google drive connection
*   etc


In [2]:
OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}

OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/sellmore-poc-category-mapping"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /
    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/category-mapping" #@param
    %cd /content/drive/MyDrive


Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive


## Preparation One: Generate config json


*   Generates recore category from authorization token
*   Output
      - **{ec name}_ec_config.json**
















In [3]:
import requests
import json

authorization = "authorization token" #@param


def main():
    # Common headers for all requests
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "ja,en-US;q=0.9,en;q=0.8",
        "authorization": authorization,
        "origin": "https://app.sellmore.cc",
        "priority": "u=1, i",
        "referer": "https://app.sellmore.cc/",
        "sec-ch-ua": '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Linux"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "x-staff-code": "SF0000000000",
    }

    try:
        # Step 1: Get the configs to extract ec_accounts
        print("Fetching configs...")
        configs_url = "https://api.recore-pos.com/v1/configs"
        response = requests.get(configs_url, headers=headers)
        response.raise_for_status()

        configs_data = response.json()
        print(f"Configs response status: {response.status_code}")

        # Extract ec_accounts from the response
        ec_accounts = configs_data.get("ec_accounts", [])
        print(f"Found {len(ec_accounts)} EC accounts:")

        for account in ec_accounts:
            print(
                f"  - ID: {account['id']}, Mall: {account['mall_name']}, Name: {account['name']}, Active: {account['is_active']}"
            )

        # Step 2: Loop through each ec_account_id and get their config
        print("\nFetching individual account configs...")
        account_configs = {}

        for account in ec_accounts:
            account_id = account["id"]
            account_name = account["name"]

            print(f"\nFetching config for account ID {account_id} ({account_name})...")

            config_url = (
                f"https://api.recore-pos.com/v1/ec/accounts/{account_id}/config"
            )

            try:
                config_response = requests.get(config_url, headers=headers)
                config_response.raise_for_status()

                config_data = config_response.json()
                account_configs[account_id] = {
                    "account_info": account,
                    "config": config_data,
                }

                print(f"  ✓ Successfully fetched config for {account_name}")
                print(
                    f"  Config keys: {list(config_data.keys()) if isinstance(config_data, dict) else 'Non-dict response'}"
                )

            except requests.exceptions.RequestException as e:
                print(f"  ✗ Error fetching config for {account_name}: {e}")
                account_configs[account_id] = {
                    "account_info": account,
                    "config": None,
                    "error": str(e),
                }

        # Step 3: Save results - both combined file and individual files
        # Save combined file
        output_file = "ec_accounts_configs.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(
                {"configs": configs_data, "account_configs": account_configs},
                f,
                indent=2,
                ensure_ascii=False,
            )

        print(f"\n✓ Combined data saved to {output_file}")

        # Save individual files for each EC account
        print("\nSaving individual EC config files...")
        for account_id, account_data in account_configs.items():
            account_info = account_data["account_info"]
            config = account_data.get("config")

            # Create filename based on account name (sanitized)
            account_name = account_info["name"].lower()
            # Replace Japanese characters and special chars with safe alternatives
            name_mapping = {
                "amazon": "amazon",
                "ヤフオク！": "yauction",
                "楽天": "rakuten",
                "shopify": "shopify",
                "ヤフーショッピング": "yshopping",
                "ラクマ": "rakuma",
                "メルカリshops": "mercari_shops",
            }

            # Use mall_id as fallback if name mapping doesn't exist
            safe_name = name_mapping.get(account_name, account_info["mall_id"].lower())
            filename = f"{safe_name}_ec_config.json"

            # Prepare data for individual file
            individual_data = {
                "account_info": account_info,
                "config": config,
                "error": account_data.get("error"),
            }

            try:
                with open(filename, "w", encoding="utf-8") as f:
                    json.dump(individual_data, f, indent=2, ensure_ascii=False)
                print(f"  ✓ {account_info['name']} config saved to {filename}")
            except Exception as e:
                print(f"  ✗ Error saving {account_info['name']} config: {e}")

        print(f"\n✓ Individual EC config files created")

        # Summary
        print(f"\nSummary:")
        print(f"- Total accounts: {len(ec_accounts)}")
        print(
            f"- Successful config fetches: {sum(1 for config in account_configs.values() if config.get('config') is not None)}"
        )
        print(
            f"- Failed config fetches: {sum(1 for config in account_configs.values() if config.get('config') is None)}"
        )

        return account_configs

    except requests.exceptions.RequestException as e:
        print(f"Error fetching configs: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None


if __name__ == "__main__":
    result = main()

    # Optional: Print a sample of the data
    if result:
        print("\nSample account config (first account):")
        first_account_id = list(result.keys())[0] if result else None
        if first_account_id:
            sample_config = result[first_account_id]
            print(f"Account: {sample_config['account_info']['name']}")
            if sample_config.get("config"):
                print(
                    f"Config preview: {json.dumps(sample_config['config'], indent=2, ensure_ascii=False)[:500]}..."
                )
            else:
                print(f"Config error: {sample_config.get('error', 'Unknown error')}")

Fetching configs...
Error fetching configs: 401 Client Error: Unauthorized for url: https://api.recore-pos.com/v1/configs
